In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd  
import pylab as pl
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.metrics import accuracy_score

In [3]:
all_features_data = pd.read_csv("/Users/suhaniagarwal/Downloads/all_features_data.csv")


In [5]:
all_features_data.columns.tolist()

['Timestamp',
 'PM2.5 (µg/m³)',
 'PM10 (µg/m³)',
 'NO (µg/m³)',
 'NO2 (µg/m³)',
 'NOx (ppb)',
 'NH3 (µg/m³)',
 'SO2 (µg/m³)',
 'CO (mg/m³)',
 'Ozone (µg/m³)',
 'Benzene (µg/m³)',
 'Toluene (µg/m³)',
 'Index Value',
 'temperature_2m_mean (°C)',
 'temperature_2m_max (°C)',
 'temperature_2m_min (°C)',
 'apparent_temperature_mean (°C)',
 'apparent_temperature_max (°C)',
 'apparent_temperature_min (°C)',
 'wind_speed_10m_max (km/h)',
 'wind_gusts_10m_max (km/h)',
 'wind_direction_10m_dominant (°)',
 'Number of Admissions']

In [7]:
y = all_features_data["Number of Admissions"]


In [8]:
# Create lagged features for days 3–5
lags = [3, 4, 5]

lag_data = all_features_data.copy()

# Columns to lag (exclude target + timestamp)
cols_to_lag = lag_data.columns.difference(
    ["Number of Admissions", "Timestamp"]
)

for lag in lags:
    X_shifted = lag_data[cols_to_lag].shift(lag)
    X_shifted.columns = [f"{col}_lag{lag}" for col in cols_to_lag]
    lag_data = pd.concat([lag_data, X_shifted], axis=1)

# Drop rows with NaNs introduced by lagging
lag_data = lag_data.dropna()

# Align target
y_lag_3_5 = y.loc[lag_data.index]



In [10]:
print(lag_data.shape)
print(y_lag_3_5.shape)



(695, 86)
(695,)


In [77]:
all_features_data

,Timestamp,PM2.5 (µg/m³),PM10 (µg/m³),NO (µg/m³),NO2 (µg/m³),NOx (ppb),NH3 (µg/m³),SO2 (µg/m³),CO (mg/m³),Ozone (µg/m³),...,temperature_2m_mean (°C),temperature_2m_max (°C),temperature_2m_min (°C),apparent_temperature_mean (°C),apparent_temperature_max (°C),apparent_temperature_min (°C),wind_speed_10m_max (km/h),wind_gusts_10m_max (km/h),wind_direction_10m_dominant (°),Number of Admissions
0,2017-05-01,38.350,61.36,16.18,5.71,116.68,50.755,1.82,0.700,25.85,...,26.9,35.6,17.4,26.3,36.0,16.9,13.7,29.5,8,28.0
1,2017-05-02,48.600,77.75,15.48,5.74,116.68,50.755,4.74,0.900,43.53,...,29.4,37.8,19.4,27.7,38.1,16.9,17.2,29.5,46,14.0
2,2017-05-03,46.940,75.10,18.65,4.80,116.68,50.755,2.38,1.190,19.42,...,30.2,36.8,25.1,28.4,35.9,23.7,19.1,40.7,39,15.0
3,2017-05-04,57.095,119.76,15.73,14.73,116.68,50.755,1.82,0.645,11.61,...,29.2,36.0,23.6,28.1,37.4,21.9,18.8,35.3,20,17.0
4,2017-05-05,75.770,121.22,8.48,8.62,116.68,50.755,0.99,0.490,32.05,...,29.5,38.0,20.9,28.6,37.7,19.5,15.3,33.8,326,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,2019-03-27,56.110,94.46,14.94,1.30,12.39,21.310,10.59,0.000,23.20,...,21.9,29.6,13.9,23.0,30.9,13.3,11.2,24.5,324,23.0
696,2019-03-28,28.690,73.86,27.43,9.46,32.22,43.840,9.62,0.000,40.74,...,23.8,30.6,18.0,25.4,32.1,18.9,11.1,25.6,332,41.0
697,2019-03-29,47.540,81.34,40.21,10.34,47.18,50.500,19.67,0.000,34.93,...,24.3,31.8,17.5,25.6,33.5,18.5,11.0,24.1,316,27.0
698,2019-03-30,52.130,88.82,65.08,10.78,66.81,66.040,37.93,0.000,36.24,...,24.0,32.2,18.3,24.6,34.9,17.1,21.7,44.6,334,13.0


In [76]:
lag_data_cat

,Timestamp,PM2.5 (µg/m³),PM10 (µg/m³),NO (µg/m³),NO2 (µg/m³),NOx (ppb),NH3 (µg/m³),SO2 (µg/m³),CO (mg/m³),Ozone (µg/m³),...,apparent_temperature_max (°C)_lag5,apparent_temperature_mean (°C)_lag5,apparent_temperature_min (°C)_lag5,temperature_2m_max (°C)_lag5,temperature_2m_mean (°C)_lag5,temperature_2m_min (°C)_lag5,wind_direction_10m_dominant (°)_lag5,wind_gusts_10m_max (km/h)_lag5,wind_speed_10m_max (km/h)_lag5,high_admission_day
5,2017-05-06,45.17,72.27,14.23,6.48,116.68,50.755,3.69,0.88,32.92,...,36.0,26.3,16.9,35.6,26.9,17.4,8.0,29.5,13.7,0
6,2017-05-07,52.40,83.85,13.72,6.77,116.68,50.755,2.15,0.83,56.53,...,38.1,27.7,16.9,37.8,29.4,19.4,46.0,29.5,17.2,1
7,2017-05-08,42.62,68.20,2.90,10.87,116.68,50.755,2.59,0.40,18.46,...,35.9,28.4,23.7,36.8,30.2,25.1,39.0,40.7,19.1,1
8,2017-05-09,74.85,119.76,3.28,12.34,116.68,50.755,3.03,0.98,9.81,...,37.4,28.1,21.9,36.0,29.2,23.6,20.0,35.3,18.8,1
9,2017-05-10,109.99,175.98,17.82,3.97,116.68,50.755,9.34,1.43,14.10,...,37.7,28.6,19.5,38.0,29.5,20.9,326.0,33.8,15.3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,2019-03-27,56.11,94.46,14.94,1.30,12.39,21.310,10.59,0.00,23.20,...,24.7,16.7,10.0,26.0,18.2,11.6,310.0,33.1,15.9,1
696,2019-03-28,28.69,73.86,27.43,9.46,32.22,43.840,9.62,0.00,40.74,...,24.7,16.7,10.1,25.6,17.9,12.1,321.0,53.6,25.5,1
697,2019-03-29,47.54,81.34,40.21,10.34,47.18,50.500,19.67,0.00,34.93,...,25.6,17.6,11.7,25.4,18.0,12.5,308.0,33.5,17.1,1
698,2019-03-30,52.13,88.82,65.08,10.78,66.81,66.040,37.93,0.00,36.24,...,26.5,18.4,13.6,25.0,18.1,13.9,323.0,23.8,9.8,0


In [11]:
X_lag = lag_data.drop(columns=["Number of Admissions", "Timestamp"])
y_lag = y_lag_3_5


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_lag, y_lag, test_size=0.2, shuffle=False
)


In [16]:
import xgboost as xgb


In [17]:
model = xgb.XGBRegressor(
    n_estimators=50,
    max_depth=4,
    learning_rate=0.05,
    random_state=42
)

model.fit(X_train, y_train)

,"objective objective: typing.Union[str, xgboost.sklearn._SklObjWProto, typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]Specify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'reg:squarederror'
,"base_score base_score: typing.Union[float, typing.List[float], NoneType]The initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.Optional[typing.List[xgboost.callback.TrainingCallback]]List of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: typing.Optional[float]Subsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: typing.Optional[float]Subsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: typing.Optional[float]Subsample ratio of columns when constructing each tree.,None
,"device device: typing.Optional[str].. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: typing.Optional[int].. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,False
,"eval_metric eval_metric: typing.Union[str, typing.List[typing.Union[str, typing.Callable]], typing.Callable, NoneType].. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import load_diabetes

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error

y_pred = model.predict(X_test)
print("R²:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))


In [78]:
y_pred = model.predict(X_test)


The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [79]:
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("MAE:", mae)
print("RMSE:", rmse)
print("R² score:", r2)

MAE: 6.3776261159126335
RMSE: 7.897704872215094
R² score: -0.13475845773415718


In [20]:
lag_data_cat = lag_data.copy()

In [21]:
threshold = 20

lag_data_cat["high_admission_day"] = (lag_data_cat["Number of Admissions"] >= threshold).astype(int)

In [22]:
X_cat_lag = lag_data_cat.drop(columns=["Number of Admissions", "high_admission_day", "Timestamp"])
y_cat_lag = lag_data_cat["high_admission_day"]

In [23]:
from sklearn.model_selection import train_test_split

X_train_cat_lag, X_test_cat_lag, y_train_cat_lag, y_test_cat_lag = train_test_split(
    X_cat_lag, y_cat_lag,
    test_size=0.2,
    shuffle=False
)

In [42]:
import xgboost as xgb

clf = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.85,
    colsample_bytree=0.85,
    scale_pos_weight=(y_train_cat_lag == 0).sum() / (y_train_cat_lag == 1).sum(),
    eval_metric="logloss",
    random_state=42
)







clf.fit(X_train_cat_lag, y_train_cat_lag)

,"objective objective: typing.Union[str, xgboost.sklearn._SklObjWProto, typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]Specify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'binary:logistic'
,"base_score base_score: typing.Union[float, typing.List[float], NoneType]The initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.Optional[typing.List[xgboost.callback.TrainingCallback]]List of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: typing.Optional[float]Subsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: typing.Optional[float]Subsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: typing.Optional[float]Subsample ratio of columns when constructing each tree.,0.85
,"device device: typing.Optional[str].. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: typing.Optional[int].. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,False
,"eval_metric eval_metric: typing.Union[str, typing.List[typing.Union[str, typing.Callable]], typing.Callable, NoneType].. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import load_diabetes 

In [43]:
y_proba = clf.predict_proba(X_test_cat_lag)[:, 1]
print("ROC–AUC:", roc_auc_score(y_test_cat_lag, y_proba))

ROC–AUC: 0.5098245614035087


y_pred_cat_lag = (y_proba_cat_lag >= 0.6).astype(int)


In [41]:
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    classification_report,
    confusion_matrix
)

y_proba_cat_lag = clf.predict_proba(X_test_cat_lag)[:, 1]
y_pred_cat_lag = (y_proba_cat_lag >= 0.5).astype(int)

print("Accuracy:", accuracy_score(y_test_cat_lag, y_pred_cat_lag))
print("ROC–AUC:", roc_auc_score(y_test_cat_lag, y_proba_cat_lag))
print(confusion_matrix(y_test_cat_lag, y_pred_cat_lag))
print(classification_report(y_test_cat_lag, y_pred_cat_lag))

Accuracy: 0.7194244604316546
ROC–AUC: 0.4747368421052631
[[ 1 24]
 [15 99]]
              precision    recall  f1-score   support

           0       0.06      0.04      0.05        25
           1       0.80      0.87      0.84       114

    accuracy                           0.72       139
   macro avg       0.43      0.45      0.44       139
weighted avg       0.67      0.72      0.69       139

